### step1：watsonx モデルへのアクセス設定

In [1]:
import os
from getpass import getpass
from llama_index.readers.file import PyMuPDFReader

watsonx_api_key = "0i-_-6pigerNnnRaU8_oiybRZz_UxMQuBHpE_copxSdw"
os.environ["WATSONX_APIKEY"] = watsonx_api_key

watsonx_project_id = "b596c884-f867-4771-afcc-f9fd10dae1a4"
os.environ["WATSONX_PROJECT_ID"] = watsonx_project_id

from llama_index.llms.ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames

rag_gen_parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "sample",
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 150,
    GenTextParamsMetaNames.TEMPERATURE: 0.5,
    GenTextParamsMetaNames.TOP_K: 5,
    GenTextParamsMetaNames.TOP_P: 0.7
}
watsonx_llm = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",  # モデル名のスペースを修正
    url="https://us-south.ml.cloud.ibm.com",
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    max_new_tokens=512,
    params=rag_gen_parameters,
)

2025-09-10 14:38:13,085 - INFO - Client successfully initialized
2025-09-10 14:38:14,682 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-10 14:38:14,682 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-10 14:38:15,285 - INFO - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
/Users/yamawakidaiki/internship/AGAIN/RAG/lib/python3.12/site-packages/ibm_watsonx_ai/f

### step2：イベントループの設定

In [2]:
import os
import gradio as gr

# ---
# api_key = os.environ.get("WATSONX_APIKEY") or "YOUR WATSONX API KEY"
# project_id = os.environ.get("WATSONX_PROJECT_ID") or "YOUR WATSONX PROJECT ID"
# ---
os.environ["WATSONX_APIKEY"] = "0i-_-6pigerNnnRaU8_oiybRZz_UxMQuBHpE_copxSdw"
os.environ["WATSONX_PROJECT_ID"] = "b596c884-f867-4771-afcc-f9fd10dae1a4"

api_key = os.environ.get("WATSONX_APIKEY")
project_id = os.environ.get("WATSONX_PROJECT_ID")

from langchain_ibm import WatsonxLLM
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames

/Users/yamawakidaiki/internship/AGAIN/RAG/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### step7：日本語LLMの導入

In [3]:
parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "sample",
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 50,
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 1024,
    GenTextParamsMetaNames.TEMPERATURE: 0.7,
    GenTextParamsMetaNames.TOP_K: 50,
    GenTextParamsMetaNames.TOP_P: 0.9,
}

try:
    model = ModelInference(
        model_id="openai/gpt-oss-120b",
        params=parameters,
        credentials={
            "apikey": os.getenv("WATSONX_APIKEY"),
            "url": "https://us-south.ml.cloud.ibm.com"
        },
        project_id=os.getenv("WATSONX_PROJECT_ID")
    )
    
    # LangChainのLLMラッパーを作成
    # Note: ModelInferenceは直接LangChainのLLMとして使えないため、
    # 応答を生成する関数を持つカスタムラッパーを作成します。
    class CustomWatsonxLLM(WatsonxLLM):
        model_inference: ModelInference

        def _call(self, prompt, stop=None, run_manager=None, **kwargs):
            messages = [{"role": "user", "content": prompt}]
            response = self.model_inference.chat(messages=messages)
            return response["choices"][0]["message"]["content"]

    llm = CustomWatsonxLLM(
        model_inference=model,
        model_id="openai/gpt-oss-120b",
        url="https://us-south.ml.cloud.ibm.com",
        project_id=project_id
    )

except Exception as e:
    raise

2025-09-10 14:38:16,842 - INFO - Client successfully initialized
2025-09-10 14:38:18,057 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-10 14:38:18,057 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-10 14:38:18,128 - INFO - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
2025-09-10 14:38:18,128 - INFO - Successfully finished Get available foundation models 

In [4]:
# checkpoint
# トークナイザが SentencePiece を使って正常ロードできるか確認
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("pkshatech/GLuCoSE-base-ja")
print(type(tok), "OK")

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="pkshatech/GLuCoSE-base-ja"  # 必要なら later に batch サイズ等を調整
)

<class 'transformers.models.mluke.tokenization_mluke.MLukeTokenizer'> OK


2025-09-10 14:38:23,203 - INFO - Load pretrained SentenceTransformer: pkshatech/GLuCoSE-base-ja


In [5]:
# 以下のコードでは、すでに別のモデルを設定していた場合でも、WatsonxLLM クラスを使って granite-3-2-8b-instruct モデルで 上書き する形になります 。
rag_gen_parameters[GenTextParamsMetaNames.MAX_NEW_TOKENS] = 512
watsonx_llm = WatsonxLLM(
	model_id="ibm/granite-3-2-8b-instruct",
	url="https://us-south.ml.cloud.ibm.com",
	project_id=os.getenv("WATSONX_PROJECT_ID"),
	params=rag_gen_parameters,
)

# トマトに関するpdfを読み込ませる。
from llama_index.readers.file import PyMuPDFReader
loader = PyMuPDFReader()
# pdf_doc_ja = loader.load(file_path="./docs/housetomato.pdf")
pdf_doc_ja = loader.load(file_path="./docs/housetomato.pdf")


from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)

index = VectorStoreIndex.from_documents(
	pdf_doc_ja, transformations=[splitter],
	embed_model=Settings.embed_model
)



# 日本語に対応した Embedding モデル に変更
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.embed_model = HuggingFaceEmbedding(
	model_name="pkshatech/GLuCoSE-base-ja"
)

# ここから追加

# 新しいドキュメントと設定でインデックスを再構築
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=512) # 日本語なのでチャンクサイズを調整
index_ja = VectorStoreIndex.from_documents(
    pdf_doc_ja, # 日本語のドキュメントを使用
    transformations=[splitter],
    embed_model=Settings.embed_model
)

# 新しいインデックスでリトリーバーを再構築
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever

vector_retriever_ja = index_ja.as_retriever(similarity_top_k=2)
bm25_retriever_ja = BM25Retriever.from_defaults(
    docstore=index_ja.docstore,
    similarity_top_k=2
)

2025-09-10 14:38:29,752 - INFO - Client successfully initialized
2025-09-10 14:38:30,671 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-10 14:38:30,671 - INFO - HTTP Request: GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200 "HTTP/1.1 200 OK"
2025-09-10 14:38:30,715 - INFO - Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2025-08-27&project_id=b596c884-f867-4771-afcc-f9fd10dae1a4&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'
2025-09-10 14:38:30,715 - INFO - Successfully finished Get available foundation models 

In [6]:
query_gen_prompt_str = (
    "あなたは、1つの入力クエリに基づいて複数の検索クエリを生成する有能なアシスタントです。\n"
    "{num_queries}個の検索クエリを、1行につき1つずつ生成してください。\n"
    "以下のクエリに関連する検索クエリを生成してください：\n"
    "\n"
    "クエリ: {query}\n"
    "検索クエリ:\n"
)


In [7]:
from llama_index.core.retrievers import QueryFusionRetriever
Settings.llm = watsonx_llm

from llama_index.retrievers.bm25 import BM25Retriever
vector_retriever = index.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
	docstore=index.docstore,
	similarity_top_k=2
)

retriever = QueryFusionRetriever(
	[vector_retriever, bm25_retriever],
	similarity_top_k=4,
	num_queries=4,  # クエリ生成を無効にする場合は1
	mode="reciprocal_rerank",
	use_async=False,
	verbose=False,
	query_gen_prompt=query_gen_prompt_str  # クエリ生成プロンプトを上書き
)

2025-09-10 14:38:38,823 - DEBUG - Building index from IDs objects


In [8]:
prompt_template = """
以下のコンテキストのみを使用して、最後の質問に答えてください。

コンテキスト:
{context}

質問:
{question}

回答:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": prompt}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Input should be a valid dictionary or instance of BaseRetriever [type=model_type, input_value=<llama_index.core.retriev...r object at 0x31da90b90>, input_type=QueryFusionRetriever]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine(retriever)

#### Gradio を使ったチャットボットの作成

In [ ]:
def chat_function(message, history):
    try:
        response = qa({"query": message})
        result = response["result"]
        
        # ソースドキュメントの情報を追加
        source_docs = response.get("source_documents")
        if source_docs:
            result += "\n\n--- ソース ---\n"
            for doc in source_docs:
                # メタデータからページ番号を取得（存在する場合）
                page_num = doc.metadata.get('page', 'N/A')
                result += f"- ページ: {page_num}\n"
                # コンテンツの最初の100文字を表示
                result += f"  内容: {doc.page_content[:100]}...\n"
                
        return result
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

demo = gr.ChatInterface(
    fn=chat_function,
    title="RAG with gpt-4o",
    description="PDFドキュメントに関する質問をしてください。",
    theme="soft",
    examples=[
        "トマトの育て方について教えてください。",
        "IBMの2023年の収益はいくらですか？",
    ]
)

2025-09-10 14:14:31,909 - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-09-10 14:14:31,916 - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


2025-09-10 14:14:32,334 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2025-09-10 14:17:50,619 - INFO - HTTP Request: POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27 "HTTP/1.1 200 OK"
2025-09-10 14:17:50,633 - INFO - Successfully finished generate for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27'
2025-09-10 14:17:56,660 - INFO - HTTP Request: POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27 "HTTP/1.1 200 OK"
2025-09-10 14:17:56,664 - INFO - Successfully finished generate for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2025-08-27'


# ベクトルデータベース
- LlamaIndex のインデックスストレージを使用して，ベクトルをRAMではなくベクトルデータベースに保存するようにする。
  - ベクトルデータベースは、**HNSW（Hierarchical Navigable Small World）**のような特殊なインデックスアルゴリズムを使って、類似したベクトルをまとめて保存
  - 例：すべての本（ベクトル）を一つの長い棚に並べるのではなく（これはブルートフォース検索）、ベクトルデータベースは階層的なシステムを使用する。本をジャンル分け（類似ベクトルごとにカテゴリ作成）=>ジャンルの中でサブジャンルや著者ごとに整理してネットワーク作成。
  - 本同士のつながりが、探している本やその近くにある類似した本まで素早く導いてくれる。これが、ベクトルデータベースが類似した埋め込み（エンベディング）を非常に効率的に見つけられる理由

### step1：DockerのDownload